<a href="https://colab.research.google.com/github/gtzan/mir_book/blob/master/discretizing_the_frequency_continuum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import math
import IPython.display as ipd

def make_sine(freq, duration, samplerate=44100):
    """Generates one cycle of a sine wave."""
    t = np.linspace(0, duration, int(samplerate * duration), endpoint=False)
    return np.sin(2 * np.pi * freq * t)

def addsyn(freq, dur, amplist, samplerate=44100):
    """Creates a waveform by summing sine waves (harmonics)."""
    t = np.linspace(0, dur, int(samplerate * dur), endpoint=False)
    output = np.zeros_like(t)
    for i, amp in enumerate(amplist):
        # Add harmonics (multiples of the base frequency)
        output += amp * make_sine(freq * (i + 1), dur, samplerate)
    # Normalize to prevent clipping
    return t, output / np.max(np.abs(output))

# Example: A simple violin-like sound with overtones
# Amplist: [Fundamental, 2nd Harm, 3rd Harm, ...]
t, violin_wave = addsyn(220, 1, [1, 0.263, 0.14, 0.099, 0.209, 0.02, 0.029, 0.077, 0.017, 0.01])

def play_freq(freq, dur):
    amplist = [1, 0.263, 0.14, 0.099, 0.209, 0.02, 0.029, 0.077, 0.017, 0.01]
    t, audio = addsyn(freq, dur, amplist, samplerate=44100)
    return audio


ipd.Audio(violin_wave, rate=44100)


In [20]:
def karplus_strong(frequency, duration, sample_rate=44100, decay=0.996):
    """
    Generates a string-like sound at a specific frequency.
    """
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    # 1. Calculate delay line length (N) based on desired frequency
    N = int(sample_rate / frequency)

    # 2. Initialize the delay line with noise (the "pluck")
    # This represents the initial energy transferred to the string
    ring_buffer = np.random.uniform(-1, 1, N)

    # 3. Generate the output signal
    num_samples = int(duration * sample_rate)
    audio = np.zeros(num_samples)

    for i in range(num_samples):
        # The current sample is the first in the buffer
        audio[i] = ring_buffer[0]

        # 4. Filter: Average the first two samples and apply decay
        # This simulates high-frequency energy loss over time
        new_sample = decay * 0.5 * (ring_buffer[0] + ring_buffer[1])

        # 5. Update the buffer (remove first, append new)
        ring_buffer = np.roll(ring_buffer, -1)
        ring_buffer[-1] = new_sample

    return t, audio

# --- Usage in Notebook ---
fs = 44100
# Play an A4 note (440 Hz) for 2 seconds
t, note_a4 = karplus_strong(440, 2, sample_rate=fs)

# Use IPython to display the player widget
ipd.Audio(note_a4, rate=fs)


def pluck_freq(freq, dur):
    t, audio = karplus_strong(freq, dur, sample_rate=44100)
    return audio


In [23]:
drone = pluck_freq(220, 6)
pentatonic_scale_freqs = [440, 493.88, 554.37, 659.25, 739.99, 880]
melody = []
for freq in pentatonic_scale_freqs:
    audio = pluck_freq(freq, 0.5)
    melody.append(audio)

for freq in pentatonic_scale_freqs[::-1]:
    audio = pluck_freq(freq, 0.5)
    melody.append(audio)
melody = np.concatenate(melody)
ipd.Audio(drone + melody, rate=44100)
